In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('scraping-382202-fa58f9c946e5.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 0 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 40060002 # 店舗ID
SPREADSHEET_KEY = '169tldtWSNH9t8iaZDbxkWv5wuQRMHzBbtE8T18c6Frg' # スプレッドシートのID
#pstart = 1 # パチンコの1台目
sstart = 1 # スロットの1台目
wait = 0.02

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"
Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
path = "C:\\Users\\garden026\\WD\\" + str(wd) + "\\chromedriver.exe"

chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

s20table = []
s5table = []

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "568":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "568":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

WebDriverException: Message: unknown error: Failed to create Chrome process.
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D6A813+48355]
	(No symbol) [0x00CFC4B1]
	(No symbol) [0x00C05358]
	(No symbol) [0x00C22026]
	(No symbol) [0x00C20579]
	(No symbol) [0x00C50C55]
	(No symbol) [0x00C5093C]
	(No symbol) [0x00C4A536]
	(No symbol) [0x00C282DC]
	(No symbol) [0x00C293DD]
	GetHandleVerifier [0x00FCAABD+2539405]
	GetHandleVerifier [0x0100A78F+2800735]
	GetHandleVerifier [0x0100456C+2775612]
	GetHandleVerifier [0x00DF51E0+616112]
	(No symbol) [0x00D05F8C]
	(No symbol) [0x00D02328]
	(No symbol) [0x00D0240B]
	(No symbol) [0x00CF4FF7]
	BaseThreadInitThunk [0x75FFD109+25]
	RtlExpandEnvironmentStrings_U [0x774424ED+493]
	RtlExpandEnvironmentStrings_U [0x774424C1+449]


In [17]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

s20table = []
s5table = []

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "568":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "568":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['07/31', '120053', '1', '4773', '-93.0', '0', '11', '63']
['07/31', '120121', '2', '1988', '-370.0', '0', '4', '32']
['07/31', '120075', '3', '1125', '463.0', '0', '2', '9']
['07/31', '120070', '4', '471', '-463.0', '0', '1', '2']
['07/31', '119940', '5', '995', '-602.0', '5', '0', '--']
['07/31', '119947', '6', '4652', '-2037.0', '0', '1', '15']
['07/31', '119955', '7', '1486', '185.0', '0', '1', '17']
['07/31', '119955', '8', '881', '-556.0', '0', '1', '5']
['07/31', '120034', '9', '6237', '741.0', '0', '10', '56']
['07/31', '120034', '10', '2920', '-2315.0', '0', '3', '11']
['07/31', '120034', '11', '3203', '278.0', '0', '5', '29']
['07/31', '120034', '12', '5863', '1157.0', '0', '10', '52']
['07/31', '120034', '13', '2074', '-556.0', '0', '4', '15']
['07/31', '120034', '14', '5862', '-463.0', '0', '9', '48']
['07/31', '120121', '15', '1746', '556.0', '0', '5', '37']
['07/31', '120121', '16', '2341', '-93.0', '0', '3', '40']
['07/31', '120100', '17', '1774', '-1019.0', '0', '7', '1

['07/31', '120108', '142', '0', '0', '0', '0', '--']
['07/31', '120108', '143', '0', '0', '0', '0', '--']
['07/31', '120108', '144', '0', '0', '0', '0', '--']
['07/31', '120108', '145', '63', '0', '0', '0', '--']
['07/31', '120034', '146', '909', '-370.0', '0', '1', '5']
['07/31', '120034', '147', '747', '417.0', '0', '2', '8']
['07/31', '120093', '148', '1887', '-694.0', '11', '0', '11']
['07/31', '120093', '149', '0', '0', '0', '0', '--']
['07/31', '120093', '150', '261', '-231.0', '0', '0', '--']
['07/31', '120109', '151', '2885', '694.0', '0', '5', '36']
['07/31', '120109', '152', '915', '-972.0', '0', '1', '3']
['07/31', '120113', '153', '12', '0', '0', '0', '--']
['07/31', '120113', '154', '868', '-648.0', '4', '0', '2']
['07/31', '120084', '155', '318', '-231.0', '0', '0', '1']
['07/31', '120037', '156', '491', '-93.0', '2', '0', '2']
['07/31', '120121', '157', '14', '0', '0', '0', '--']
['07/31', '120051', '158', '746', '-602.0', '0', '2', '3']
['07/31', '119940', '159', '1227'

['07/31', '119916', '281', '2847', '648.0', '13', '8', '--']
['07/31', '119916', '282', '917', '-93.0', '3', '3', '--']
['07/31', '119916', '283', '5561', '1713.0', '29', '17', '--']
['07/31', '119916', '284', '2547', '370.0', '11', '5', '--']
['07/31', '119916', '285', '103', '-93.0', '0', '0', '--']
['07/31', '119916', '286', '5791', '648.0', '22', '17', '--']
['07/31', '119916', '287', '3024', '1574.0', '15', '10', '--']
['07/31', '119916', '288', '1962', '-694.0', '3', '7', '--']
['07/31', '119916', '289', '197', '-231.0', '0', '0', '--']
['07/31', '119916', '290', '1816', '-93.0', '7', '4', '--']
['07/31', '119916', '291', '296', '-185.0', '1', '0', '--']
['07/31', '119916', '292', '1499', '-93.0', '4', '7', '--']
['07/31', '119916', '293', '4660', '972.0', '19', '18', '--']
['07/31', '119916', '294', '1457', '-741.0', '2', '5', '--']
['07/31', '119916', '295', '1183', '-417.0', '4', '1', '--']
['07/31', '119916', '296', '509', '-231.0', '1', '0', '--']
['07/31', '119916', '297', 

['07/31', '120034', '415', '3474', '1389.0', '0', '7', '35']
['07/31', '120034', '416', '667', '-648.0', '0', '0', '2']
['07/31', '120034', '417', '2181', '1204.0', '0', '4', '23']
['07/31', '120034', '418', '1106', '-694.0', '0', '1', '5']
['07/31', '120034', '419', '55', '0', '0', '0', '--']
['07/31', '120077', '420', '6326', '-694.0', '0', '16', '38']
['07/31', '120077', '421', '4105', '-231.0', '0', '15', '34']
['07/31', '120077', '422', '1592', '-833.0', '0', '1', '6']
['07/31', '120077', '423', '0', '0', '0', '0', '--']
['07/31', '120077', '424', '6376', '2593.0', '0', '31', '64']
['07/31', '120077', '425', '3699', '3657.0', '0', '26', '53']
['07/31', '120077', '426', '1174', '1250.0', '0', '14', '18']
['07/31', '120077', '427', '1130', '-1296.0', '0', '0', '1']
['07/31', '120077', '428', '2715', '-2037.0', '0', '6', '12']
['07/31', '120073', '429', '2086', '-1389.0', '0', '7', '10']
['07/31', '120073', '430', '2841', '-2037.0', '0', '12', '13']
['07/31', '120073', '431', '1424',

['07/31', '120129', '550', '2787', '-880.0', '0', '12', '15']
['07/31', '120103', '551', '4234', '3750.0', '61', '4', '14']
['07/31', '120103', '552', '0', '0', '0', '0', '--']
['07/31', '120103', '553', '397', '-139.0', '2', '2', '2']
['07/31', '120103', '554', '2104', '972.0', '26', '1', '4']
['07/31', '120103', '555', '878', '-139.0', '4', '0', '4']
['07/31', '120103', '556', '137', '648.0', '4', '0', '1']
['07/31', '120103', '557', '554', '417.0', '10', '1', '2']
['07/31', '120103', '558', '0', '0', '0', '0', '--']
['07/31', '120103', '559', '3831', '-4259.0', '2', '7', '14']
['07/31', '120103', '560', '1904', '-1296.0', '3', '2', '6']
['07/31', '120103', '561', '793', '-278.0', '3', '1', '4']
['07/31', '120103', '562', '2991', '-2037.0', '6', '4', '9']
['07/31', '120103', '563', '1055', '-370.0', '3', '0', '2']
['07/31', '120103', '564', '1833', '-139.0', '8', '1', '7']
['07/31', '120103', '565', '0', '0', '0', '0', '--']
['07/31', '120103', '566', '4403', '-2546.0', '15', '8', '1

{'spreadsheetId': '169tldtWSNH9t8iaZDbxkWv5wuQRMHzBbtE8T18c6Frg',
 'updates': {'spreadsheetId': '169tldtWSNH9t8iaZDbxkWv5wuQRMHzBbtE8T18c6Frg',
  'updatedRange': 'S5RAW!F1902:M1981',
  'updatedRows': 80,
  'updatedColumns': 8,
  'updatedCells': 640}}